In [ ]:
from dotenv import load_dotenv
import os
import numpy as np 
import pandas as pd
import pdfplumber

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool


load_dotenv()

In [ ]:
# LLM definition
model = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
# 3 jobs

# cust_data = {
#   "id": 5757,
#   "template": "A",
#   "image": "",
#   "language_selected": "Bahasa Malaysia",
#   "picture": "",
#   "name": "Nadira Aisyah",
#   "adress": "No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur",
#   "email": "nadira.aisyah@techmail.com",
#   "telephone": "+60 12 345 6789",
#   "title": "Pembangun Web Asas",
#   "about": "Pembangun web dengan sedikit pengalaman dalam projek-projek asas dan tugasan berkumpulan.",
#   "language": "bahasa malaysia professional, English professional", 
#   "experience": "Company FusionTech Solutions, Jurutera Perisian, Kuala Lumpur, 2021 – Kini. Menyokong pasukan dalam tugas pembangunan harian, membantu membetulkan isu kecil dalam aplikasi web, menghadiri mesyuarat mingguan dan membuat dokumentasi asas projek. Company InnovaCode, Pembangun Web Junior, Selangor, 2019 – 2021. Membantu dalam mereka bentuk halaman web asas, membuat kemas kini kandungan laman web secara berkala, menjalankan ujian ringkas sebelum penghantaran. Company DataMinds, Intern Analisis Data, Cyberjaya, 2018 – 2019. Membantu memindahkan data ke dalam Excel, menyediakan laporan asas dengan bantuan penyelia, mempelajari asas pengendalian dataset kecil.",
#   "education": "Ijazah Sarjana Muda Sains Komputer, Universiti Malaya, 2015 – 2019. CGPA 2.80/4.00. STPM Sains, Kolej Tingkatan Enam Cheras, 2013 – 2015. Lulus semua subjek. SPM, SMK Seri Bintang Utara, 2012. 5 kredit termasuk Matematik.",
#   "strength": "able to communicate well in a team environment, work hard and good time management",
#   "reference": "Pn. Farah Lina, Pegawai Pentadbiran di FusionTech Solutions, farah.lina@fusiontech.com, +60 12 678 9012. En. Ahmad Zulkifli, Pembantu Pensyarah di Universiti Malaya, ahmad.zulkifli@um.edu.my, +60 13 123 4567.",
#   "technical_skills": "Python 30%, JavaScript 20%, SQL 15%, React 25%.",
#   "soft_skills": "Komunikasi 20%, Kerja Berpasukan 21%, Pengurusan Masa 19%, Kepimpinan 18%.",
#   "certification": "Coursesera Asas Python - 2023, Coursesera Pengenalan Cloud - 2022."
# }

# 1 jobs


cust_data = {
  "id": 5757,
  "template": "A",
  "image": "",
  "language_selected": "English",
  "picture": "",
  "name": "Nadira Aisyah",
  "adress": "No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur",
  "email": "nadira.aisyah@techmail.com",
  "telephone": "+60 12 345 6789",
  "title": "Pembangun Web Asas",
  "about": "Pembangun web asas dengan pengalaman terhad dalam tugasan teknikal ringan. Masih membina kemahiran dalam pengaturcaraan dan reka bentuk web moden.",
  "language": "bahasa malaysia professional, English professional",
  "experience": "",
  "education": "Diploma Teknologi Maklumat, Politeknik Shah Alam, 2018 – 2021. CGPA 2.65/4.00.",
  "strength": "able to communicate well in a team environment, work hard and good time management",
  "reference": "Pn. Farah Lina, Pegawai Pentadbiran di FusionTech Solutions, farah.lina@fusiontech.com, +60 12 678 9012. En. Ahmad Zulkifli, Pembantu Pensyarah di Universiti Malaya, ahmad.zulkifli@um.edu.my, +60 13 123 4567.",
  "technical_skills": "Python 30%, JavaScript 20%, SQL 15%, React 25%.",
  "soft_skills": "Komunikasi 20%, Kerja Berpasukan 21%, Pengurusan Masa 19%, Kepimpinan 18%.",
  "certification": "Coursesera Asas Python - 2023, Coursesera Pengenalan Cloud - 2022."
}

# 2 jobs


# cust_data = {
#   "id": 5757,
#   "template": "A",
#   "image": "",
#   "language_selected": "English",
#   "picture": "",
#   "name": "Nadira Aisyah",
#   "adress": "No. 12, Jalan Bunga Raya, 50450 Kuala Lumpur",
#   "email": "nadira.aisyah@techmail.com",
#   "telephone": "+60 12 345 6789",
#   "title": "Pembangun Web Asas",
#   "about": "Pembangun web asas dengan pengalaman dalam tugas teknikal ringan dan sokongan pembangunan. Mempunyai asas kukuh dalam HTML, CMS dan penyelesaian masalah laman web.",
#   "language": "bahasa malaysia professional, English professional",
#   "experience": "Company TechNova, Pembantu Pembangunan Web, Petaling Jaya, 2022 – Kini. Menyunting teks HTML asas, mengemas kini pautan rosak, dan memuat naik imej ke laman web menggunakan CMS. Company CodeWave Solutions, Intern Web Developer, Shah Alam, 2021 – 2022. Membantu dalam ujian laman web, menyelaras kandungan dengan pasukan reka bentuk, dan memantau prestasi asas laman melalui Google Analytics.",
#   "education": "Diploma Teknologi Maklumat, Politeknik Shah Alam, 2018 – 2021. CGPA 2.65/4.00.",
#   "strength": "able to adapt quickly to new technologies, solve basic website issues independently, and support senior developers effectively",
#   "reference": "Pn. Farah Lina, Pegawai Pentadbiran di FusionTech Solutions, farah.lina@fusiontech.com, +60 12 678 9012. En. Ahmad Zulkifli, Pembantu Pensyarah di Universiti Malaya, ahmad.zulkifli@um.edu.my, +60 13 123 4567.",
#   "technical_skills": "Python 30%, JavaScript 20%, SQL 15%, React 25%.",
#   "soft_skills": "Komunikasi 20%, Kerja Berpasukan 21%, Pengurusan Masa 19%, Kepimpinan 18%.",
#   "certification": "Coursesera Asas Python - 2023, Coursesera Pengenalan Cloud - 2022."
# }



# Data Ingestion

In [ ]:
id_data = cust_data["id"]
template_data = cust_data["template"]
language_data = cust_data["language_selected"]
image_data = cust_data["image"]

cust_data



# Data Transformation

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import json


instructions = """

Your task:
1. Clean and restructure the given JSON object into a clean, well-structured format as described.
2. Parse any concatenated string data into appropriate objects or arrays (e.g., experience, education, skills, strength).

Formatting Rules:
- Do NOT include inline explanations or metadata. Return **only valid JSON**.
- Do NOT return skill arrays. Instead, return a `skills` object split into `technical skills` and `soft skills`.
- All skill proficiencies must be integers only (no %, no quotes).

Required Structure:
- id: maintain as single string, 
- title: maintain as single string,
- image: maintain as single string,  
- name: maintain as single string, 
- adress: maintain as single string, 
- email: maintain as single string,  
- telephone: maintain as single string,  
- about: maintain as single string,
- language: list of objects, each containing a single key-value pair where the key is the language (string) and the value is the proficiency level (string), e.g., [{ "English": "professional" }]
- experience: list of objects with `company`, `title`, `location`, `duration`, and `details`,`details` will be a list of items
- number of jobs: list number of objects in 'experience' as integer
- education: list of objects with `level`, `institution`, `duration`, and `details`
- strength: list with bullet points
- reference: list of objects with `name`, `position`, `company`, `email`, `telephone`
- skills: object with `technical skills` and `soft skills`, each with skill-proficiency pairs
- certification: list of certification strings

"""

prompt_template = """
You are a professional assistant that structures and enhances resume details in JSON format.
You will receive a raw JSON object representing a resume and instructions.
Return only the enhanced and properly structured JSON object. No explanation.

Instructions:
{instructions}

Resume JSON:
{resume_json}
"""


resume_json_str = json.dumps(cust_data)
prompt = prompt_template.format(instructions=instructions, resume_json=resume_json_str)
message = HumanMessage(content=prompt)
response = model.invoke([message])

# Cleanup
response_str = response.content
def extract_json_from_response(response_str: str) -> str:
    if response_str.startswith("```json"):
        response_str = response_str[len("```json"):].strip()
    elif response_str.startswith("```"):
        response_str = response_str[len("```"):].strip()
    if response_str.endswith("```"):
        response_str = response_str[:-3].strip()
    return response_str

clean_json_str = extract_json_from_response(response_str)

try:
    json_object = json.loads(clean_json_str)
    print("✅ Parsed JSON successfully!")
except json.JSONDecodeError as e:
    print("❌ Failed to parse JSON:", e)
    json_object = None

In [ ]:
with open("raw_data.txt", "w") as file:
    json.dump(json_object, file, indent=2)

# Prompt chaining

In [ ]:
import json
from langchain import OpenAI, PromptTemplate, LLMChain


# 1) Enhance experience

def enhance_experience(resume: dict, llm) -> dict:
    n = resume.get("number of jobs", 0)
    target = 8 if n == 1 else 4 if n == 2 else 3

    prompt_tpl = PromptTemplate(
        input_variables=["company", "title", "existing", "target"],
        template="""
        You are a professional resume writer.
        Company: {company}
        Title: {title}
        Existing bullets:
        {existing}

        Produce exactly {target} bullet points.
        - If more than {target}, keep the strongest.
        - If fewer, invent clear, measurable achievements.

        Return ONLY a JSON array of strings.
        Example: ["Did X", "Improved Y", "Built Z"]
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt_tpl)

    for exp in resume["experience"]:
        existing_md = "\n".join(f"- {b}" for b in exp.get("details", []))

        try:
            raw = chain.run({
                "company": exp["company"],
                "title": exp["title"],
                "existing": existing_md,
                "target": target
            }).strip()

            if not raw.startswith("["):
                json_start = raw.find("[")
                json_end = raw.rfind("]") + 1
                raw = raw[json_start:json_end]

            exp["details"] = json.loads(raw)

        except json.JSONDecodeError as e:
            print(f"Failed to decode LLM response for {exp['title']}:\n{raw}\n")
            raise e

    return resume


# 2) Enhance strength section

def enhance_strength(resume: dict, llm) -> dict:
    n = resume.get("number of jobs", 0)

    if n == 0:
        prompt = PromptTemplate(
            input_variables=["title"],
            template="""
            You are a resume builder AI.
            The candidate has no job experience.
            Generate 8 professional-strength bullet points tailored to this job title:

            Title: {title}

            Return only a JSON array of 8 short strengths.
            """
                    )
    else:
        prompt = PromptTemplate(
                    input_variables=["title"],
                    template="""
                    You are a resume assistant.
                    Summarize the candidate's professional strengths in 1 item only (max 60 words), based on this role:

                    Title: {title}

                    Return only the one sentence in a JSON array.
                    """
        )

    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run({"title": resume.get("title", "")}).strip()

    try:
        if not result.startswith("["):
            json_start = result.find("[")
            json_end = result.rfind("]") + 1
            result = result[json_start:json_end]

        resume["strength"] = json.loads(result)

    except json.JSONDecodeError:
        print(f"Failed to parse strength JSON:\n{result}")
        raise

    return resume


# 3) Enhance both sections

enhanced = enhance_experience(json_object, model)
enhanced_cust_data = enhance_strength(enhanced, model)

enhanced_cust_data


# 4) Save to file

with open("enhanced_resume.txt", "w", encoding="utf-8") as f:
    json.dump(enhanced_cust_data, f, ensure_ascii=False, indent=2)

print("✅ Saved with enhanced experience and strength section")


# Data QA

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import json

# language selected
if language_data == "English": 
    language_selected = "templates English"
    instructions = """
        You are a professional resume assistant.
        Your role is to enhance and structure resume details in JSON format strictly in **English**.

        TASK OVERVIEW:
        - Receive a raw resume as a JSON object.
        - Return only the enhanced and well-structured JSON. Do **not** include explanations or extra text.

        RULES:
        - Sort job experience by latest first.
        - Use date format: DD/MM/YYYY.
        - Ensure all experiences and strengths include **measurable results or metrics**.
        - If any information is missing, fill with realistic dummy data.
        - Ensure the "about" section is written in a polished, professional tone.
    """
else:
    language_selected = "templates Bahasa Malaysia"
    instructions = """
        Anda ialah pembantu resume profesional.
        Tugas anda ialah menstruktur dan memperkemas maklumat resume dalam format JSON secara eksklusif dalam **Bahasa Malaysia**.

        TUGAS:
        - Terima input resume mentah dalam format JSON.
        - Pulangkan hanya objek JSON yang telah diperkemas dan distruktur semula. Jangan berikan penjelasan atau teks tambahan.

        PERATURAN:
        - Susun pengalaman kerja dari yang terkini dahulu.
        - Gunakan format tarikh: DD/MM/YYYY.
        - Pastikan semua pengalaman dan kekuatan mengandungi hasil atau metrik yang boleh diukur.
        - Jika ada maklumat yang hilang, isikan dengan data dummy yang munasabah.
        - Pastikan seksyen "tentang" ditulis secara profesional dan kemas.
    """


# template selected
format = "NonATS"
template_selected = ""

if template_data == "A": 
    format = "ATS"
    template_selected = "template_A.html"
    
elif template_data == "B":
    template_selected = "template_B.html"

elif template_data == "C":
    template_selected = "template_C.html"



prompt_template = """

Instructions:
{instructions}

Resume JSON:
{resume_json}
"""


resume_json_str = json.dumps(enhanced_cust_data)
prompt = prompt_template.format(instructions=instructions, resume_json=resume_json_str)
message = HumanMessage(content=prompt)
response = model.invoke([message])

# Cleanup
response_str = response.content
def extract_json_from_response(response_str: str) -> str:
    if response_str.startswith("```json"):
        response_str = response_str[len("```json"):].strip()
    elif response_str.startswith("```"):
        response_str = response_str[len("```"):].strip()
    if response_str.endswith("```"):
        response_str = response_str[:-3].strip()
    return response_str

clean_json_str = extract_json_from_response(response_str)

try:
    enhanced_cust_data = json.loads(clean_json_str)
    print("✅ Parsed JSON successfully!")
except json.JSONDecodeError as e:
    print("❌ Failed to parse JSON:", e)
    enhanced_cust_data = None

In [ ]:
with open("enhance_data.txt", "w") as file:
    json.dump(enhanced_cust_data, file, indent=2)

In [ ]:
from jinja2 import Environment, FileSystemLoader

# Set up Jinja2 environment
env = Environment(loader=FileSystemLoader("."))  # current directory

# direct to temp selected
template = env.get_template(f"./{language_selected}/{template_selected}")

# Render with data
output_html = template.render(**json_object)

# Write to output file
with open(f"./generated html/{id_data}_output_resume.html", "w", encoding="utf-8") as f:
    f.write(output_html)